# League of Legends Match Predictor

This notebook provides an interactive environment for exploring the dataset, training the logistic regression model, and visualizing the results.

In [ ]:
# Import necessary libraries
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from src.data_loader import load_data
from src.model import LogisticRegressionModel
from src.train import train_model
from src.evaluate import evaluate_model
from src.visualize import plot_confusion_matrix, plot_roc_curve
from src.feature_importance import plot_feature_importance

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# Load and preprocess the data
data = load_data('data/league_of_legends_data_large.csv')
X = data.drop('win', axis=1)
y = data['win']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32).to(device)
X_test = torch.tensor(X_test, dtype=torch.float32).to(device)
y_train = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1).to(device)
y_test = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1).to(device)

In [ ]:
# Initialize the model
input_dim = X_train.shape[1]
model = LogisticRegressionModel(input_dim).to(device)

# Train the model
train_model(model, X_train, y_train, epochs=1000, learning_rate=0.01)


In [ ]:
# Evaluate the model
train_acc, test_acc = evaluate_model(model, X_train, y_train, X_test, y_test)
print(f'Training Accuracy: {train_acc:.4f}')
print(f'Test Accuracy: {test_acc:.4f}')

In [ ]:
# Visualize the results
plot_confusion_matrix(model, X_test, y_test)
plot_roc_curve(model, X_test, y_test)


In [ ]:
# Analyze feature importance
plot_feature_importance(model, X.columns)